# Replication assignment

Replicate the results found in [Bryant et al  (2010)](https://doi.org/10.1016/j.techfore.2009.08.002), [Rozenberg et al (2014)](https://link.springer.com/content/pdf/10.1007/s10584-013-0904-3.pdf), and [Hamarat et al (2013)](https://doi.org/10.1016/j.techfore.2012.10.004). The data files are named accordingly. You can also look at [Kwakkel & Jaxa-Rozen (2016)](https://www.sciencedirect.com/science/article/pii/S1364815215301092), which gives a succint summary of these three cases. 

An additional useful, if dated, resource for Bryant et al is https://waterprogramming.wordpress.com/2015/08/05/scenario-discovery-in-python/ 



# Bryant & Lempert

Load the data using pandas. The experiments are in colums 2-10, while the outcome is in column 15. If you look at the paper carefully, you will see that they change the peel_alpha in PRIM. This controls the leniency of the algorithm.


In [ ]:
import pandas as pd 

df_bryant = pd.read_csv(r".\data\Bryant et al 2010.csv", index_col=False)

df_bryant.head()

experiments = df_bryant.iloc[:, 2:11]
outcome = df_bryant.iloc[:, 15].values




experiments.head()






# Rozenberg et al


* The experiments are the first seven columns. The remainder specify whether a given experiment belongs to a given SSP.
* Apply prim separately for each SSP
* you have to play with the threshold for each SSP, the default of 0.8 is at times too high
* if you look at the paper carefully, you will discover that they change some of the default parameters of PRIM


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

from ema_workbench.analysis import prim
from ema_workbench.util import ema_logging

ema_logging.log_to_stderr(ema_logging.INFO)



df_rozen = pd.read_csv(r".\data\Rozenberg et al 2014.csv",delimiter=',')



experiments = df_rozen.iloc[:, 0:7]
outcome_1 = df_rozen.iloc[:,7].values
outcome_2 = df_rozen.iloc[:,8].values
outcome_3 = df_rozen.iloc[:,9].values
outcome_4 = df_rozen.iloc[:,10].values
outcome_5 = df_rozen.iloc[:,11].values



prim_alg_1 = prim.Prim(experiments, outcome_1, threshold=0.4, peel_alpha=0.1)
box1 = prim_alg_1.find_box()

box1.show_tradeoff()
print(box1.resample(5))
box1.inspect(5)
box1.inspect(5, style='graph')

plt.show()








In [ ]:

prim_alg_2 = prim.Prim(experiments, outcome_2, threshold=0.4, peel_alpha=0.1)
box2 = prim_alg_2.find_box()

box2.show_tradeoff()
print(box2.resample(5))
box2.inspect(5)
box2.inspect(5, style='graph')

plt.show()

In [ ]:
prim_alg_3 = prim.Prim(experiments, outcome_3, threshold=0.2, peel_alpha=0.1)
box3 = prim_alg_3.find_box()

box3.show_tradeoff()
print(box3.resample(4))
box3.inspect(4)
box3.inspect(4, style='graph')

plt.show()

In [ ]:

from ema_workbench.analysis import prim
prim_alg_4 = prim.Prim(experiments, outcome_4, threshold=0.3, peel_alpha=0.1)
box4 = prim_alg_4.find_box()

box4.show_tradeoff()
print(box4.resample(2))
box4.inspect(2)
box4.inspect(2, style='graph')

plt.show()

# Hamarat et al

this is the most difficult dataset to analyze with prim. It has been generated using the ema_workbench and can be loaded with the `load_results` function. Experiments are of interest if the end state for `'fraction renewables'` is lower than the initial value.

* PRIM struggles with finding a good explanation for this case, why?
* [Dalal et al (2013)](https://www.sciencedirect.com/science/article/pii/S1364815213001345) suggest using PCA preprocessing. The ema_workbench supports this with the `pca_preprocess` function. See also examples/prim_PCA_preprocess.py. Does PCA preprocessing makes a difference for this case?


In [ ]:
import matplotlib.pyplot as plt

from ema_workbench import ema_logging, load_results
import ema_workbench.analysis.prim as prim
ema_logging.log_to_stderr(level=ema_logging.INFO)


x, outcomes = load_results(r'.\data\Hamarat et al 2013.gz')



# specify y
y = outcomes['fraction renewables'][:, -1] 




In [ ]:


y = outcomes['fraction renewables'][:, -1] 




rotated_experiments, rotation_matrix = prim.pca_preprocess(x, y,
                                                           exclude=['model', 'policy'])

# perform prim on modified results tuple
prim_obj = prim.Prim(rotated_experiments, y, threshold=0.8)
box = prim_obj.find_box()

box.show_tradeoff()
box.inspect(22)
plt.show()